# Notes

Make sure you've created the _.env_ file and updated the __MONGODB_URI__ as well as the __SHARED_LIB_PATH__ before you start running this notebook.

This notebook uses *local* encryption mode which is not recommended in a production environment and should only be used for learning purposes.

# Assign Your Application Variables

The code samples in this tutorial use the following variables to perform the Queryable Encryption workflow:

 - kms_provider_name - The KMS you're using to store your Customer Master Key. Set this variable to "local" for this tutorial.

 - uri - Your MongoDB deployment connection URI. Set your connection URI in the MONGODB_URI environment variable or replace the value directly.

 - key_vault_database_name - The database in MongoDB where your data encryption keys (DEKs) will be stored. Set this variable to "encryption".

 - key_vault_collection_name - The collection in MongoDB where your DEKs will be stored. Set this variable to "__keyVault".

 - key_vault_namespace - The namespace in MongoDB where your DEKs will be stored. Set this variable to the values of the key_vault_database_name and key_vault_collection_name variables, separated by a period.

 - encrypted_database_name - The database in MongoDB where your encrypted data will be stored. Set this variable to "medicalRecords".

 - encrypted_collection_name - The collection in MongoDB where your encrypted data will be stored. Set this variable to "patients".

You can declare these variables by using the following code:

In [ ]:
from pymongo import MongoClient, ASCENDING
from pymongo.encryption import (ClientEncryption)
from pymongo.encryption_options import AutoEncryptionOpts
from bson.codec_options import CodecOptions
from bson.binary import STANDARD, UUID
import os
from pprint import pprint
from dotenv import load_dotenv

load_dotenv()

kms_provider_name = "local"

uri = os.environ['MONGODB_URI']  # Your connection URI

key_vault_database_name = "encryption"
key_vault_collection_name = "__keyVault"
key_vault_namespace = f"{key_vault_database_name}.{key_vault_collection_name}"
encrypted_database_name = "medicalRecords"
encrypted_collection_name = "patients"

# Create a Customer Master Key

You must create a Customer Master Key (CMK) to perform Queryable Encryption.

Create a 96-byte Customer Master Key and save it to your filesystem as the file *customer-master-key.txt*:

In [ ]:
path = "customer-master-key.txt"
file_bytes = os.urandom(96)
with open(path, "wb") as f:
    f.write(file_bytes)

# Retrieve the Customer Master Key and Specify KMS Provider Settings

Retrieve the contents of the Customer Master Key file that you generated in the previous step.

Pass the CMK value to your KMS provider settings. The client uses these settings to discover the CMK. Set the provider name to local to inform the driver you are using a Local Key Provider.

In [ ]:
path = "./customer-master-key.txt"
with open(path, "rb") as f:
    local_master_key = f.read()
    kms_provider_credentials = {
        "local": {
            "key": local_master_key
        },
    }

# Set Your Automatic Encryption Options

Create an AutoEncryptionOpts object that contains the following options:

 - The kms_provider_credentials object, defined in the previous step

 - The namespace of your Key Vault collection

 - The path to your Automatic Encryption Shared Library

In [ ]:
auto_encryption_options = AutoEncryptionOpts(
    kms_provider_credentials,
    key_vault_namespace,
    crypt_shared_lib_path=os.environ['SHARED_LIB_PATH'] # Path to your Automatic Encryption Shared Library>
)

# Create a Client to Set Up an Encrypted Collection

To create a client used to encrypt and decrypt data in your collection, instantiate a new _MongoClient_ by using your connection URI and your automatic encryption options.

In [ ]:
encrypted_client = MongoClient(uri, auto_encryption_opts=auto_encryption_options)

# Specify Fields to Encrypt

To encrypt a field, add it to the encryption schema. To enable queries on a field, add the "queries" property. Create the encryption schema as follows:

In [ ]:
encrypted_fields_map = {
    "fields": [
        {
            "path": "patientRecord.ssn",
            "bsonType": "string",
            "queries": [{"queryType": "equality"}]
        },
        {
            "path": "patientRecord.billing",
            "bsonType": "object",
        }
    ]
}

# Create the Collection

Instantiate _ClientEncryption_ to access the API for the encryption helper methods.

In [ ]:
client_encryption = ClientEncryption(
    kms_providers=kms_provider_credentials,
    key_vault_namespace=key_vault_namespace,
    key_vault_client=encrypted_client,
    codec_options=CodecOptions(uuid_representation=STANDARD)
)

Because you are using a local Customer Master Key, you don't need to provide Customer Master Key credentials. Create a variable containing an empty object to use in place of credentials when you create your encrypted collection.

In [ ]:
customer_master_key_credentials = {}

Drop the collection first in case it already exists. (Prevents an error if you are re-running this notebook)

In [ ]:
encrypted_client[encrypted_database_name].drop_collection(encrypted_collection_name)

Create your encrypted collection by using the encryption helper method accessed through the ClientEncryption class. This method automatically generates data encryption keys for your encrypted fields and creates the encrypted collection:

In [ ]:
client_encryption.create_encrypted_collection(
    encrypted_client[encrypted_database_name],
    encrypted_collection_name,
    encrypted_fields_map,
    kms_provider_name,
    customer_master_key_credentials,
)

# Insert a Document with Encrypted Fields


Insert a Document with Encrypted Fields
Create a sample document that describes a patient's personal information. Use the encrypted client to insert it into the _patients_ collection, as shown in the following example:

In [ ]:
patient_document = {
    "patientName": "Jon Doe",
    "patientId": 12345678,
    "patientRecord": {
        "ssn": "987-65-4320",
        "billing": {
            "type": "Visa",
            "number": "4111111111111111",
        },
    },
}

encrypted_collection = encrypted_client[encrypted_database_name][encrypted_collection_name]

result = encrypted_collection.insert_one(patient_document)

# Query on an Encrypted Field

The following code sample executes a find query on an encrypted field and prints the decrypted data:

In [ ]:
find_result = encrypted_collection.find_one({
    "patientRecord.ssn": "987-65-4320"
})

pprint(find_result)

The output of the preceding code sample should look similar to the following:

```json
{
  "_id": {
    "$oid": "648b384a722cb9b8392df76a"
  },
  "name": "Jon Doe",
  "record": {
    "ssn": "987-65-4320",
    "billing": {
      "type": "Visa",
      "number": "4111111111111111"
    }
  },
  "__safeContent__": [
    {
      "$binary": {
        "base64": "L1NsYItk0Sg+oL66DBj6IYHbX7tveANQyrU2cvMzD9Y=",
        "subType": "00"
      }
    }
  ]
}
```